In [1009]:
import pandas as pd
import numpy as np
import math as m1

In [1010]:
def genNodes(fileName):
    arcs = pd.read_csv(fileName)
    nodes = {}
    
    for node1, row in arcs.iterrows():
        for i in range(1,len(row)):
            if node1 in nodes and row[i] == 1 and node1 != i-1:
                nodes[node1].append(i-1)
                if i-1 in nodes:
                    nodes[i-1].append(node1)
                else:
                    nodes[i-1] = [node1]
            else:
                if row[i] == 1 and i-1 != node1:
                    nodes[node1] = [i-1]
                    if i-1 in nodes:
                        nodes[i-1].append(node1)
                    else:
                        nodes[i-1] = [node1]
    
    for node in nodes:
        temp = set(nodes[node])
        temp2 = list(temp)
        nodes[node] = temp2
    
    return nodes
    
nodes = genNodes('storeModelArcs.csv')
# print(nodes)

In [1011]:
# print(nodes)

In [1012]:
def genCosts(fileName):
    distances = pd.read_csv(fileName)
    costs = {}

    for _, row in distances.iterrows():
        node1 = row['Start Node']
        node2 = row['End Node']
        cost = row['Cost (ft)']
    
        costs[(node1, node2)] = cost
        costs[(node2, node1)] = cost
    
    return costs

costs = genCosts('storeModelArcLengths.csv')

In [1013]:
print(costs)

{(0, 0): 0, (0, 1): 12, (1, 0): 12, (1, 1): 0, (1, 2): 9, (2, 1): 9, (1, 10): 18, (10, 1): 18, (2, 2): 0, (2, 3): 9, (3, 2): 9, (2, 12): 24, (12, 2): 24, (2, 37): 0, (37, 2): 0, (3, 3): 0, (3, 4): 12, (4, 3): 12, (3, 14): 24, (14, 3): 24, (3, 36): 0, (36, 3): 0, (4, 4): 0, (4, 5): 12, (5, 4): 12, (4, 16): 24, (16, 4): 24, (4, 35): 0, (35, 4): 0, (5, 5): 0, (5, 6): 12, (6, 5): 12, (5, 18): 24, (18, 5): 24, (5, 34): 0, (34, 5): 0, (6, 6): 0, (6, 7): 9, (7, 6): 9, (6, 20): 24, (20, 6): 24, (6, 33): 12, (33, 6): 12, (7, 7): 0, (7, 8): 6, (8, 7): 6, (7, 22): 24, (22, 7): 24, (7, 32): 12, (32, 7): 12, (8, 8): 0, (8, 23): 24, (23, 8): 24, (8, 31): 12, (31, 8): 12, (9, 9): 0, (9, 10): 12, (10, 9): 12, (10, 10): 0, (10, 11): 12, (11, 10): 12, (11, 11): 0, (11, 12): 9, (12, 11): 9, (11, 24): 24, (24, 11): 24, (12, 12): 0, (12, 13): 3, (13, 12): 3, (13, 13): 0, (13, 14): 6, (14, 13): 6, (13, 25): 24, (25, 13): 24, (14, 14): 0, (14, 15): 6, (15, 14): 6, (15, 15): 0, (15, 16): 6, (16, 15): 6, (15, 

In [1014]:
def genItems(fileName):
    df = pd.read_csv(fileName)
    nodes = {}
    
    for node1, row in df.iterrows():
        start = row['Start Node']
        end = row['End Node']
        description = row['Description']
        descriptions = description.split(',')
        for item in descriptions:
            cleaned = item.strip()
            if cleaned not in nodes:
                nodes[cleaned] = [(start, end)]
            else:
                nodes[cleaned].append((start, end))
                
    return nodes
                
# nodes = genNodes('storeModelArcs.csv')
items = genItems('storeModelNodeDescriptions.csv')
# print(items)

In [1015]:
def genOddNodes(nodeList):
    oddNodes = set()
    for node in nodeList.keys():
        edges = len(nodeList[node])
        
        if (edges)%2 == 0:
            pass
        else:
            oddNodes.add(node)
    
    return oddNodes

In [1016]:
odds = genOddNodes(nodes)
print(odds)
print(len(odds))

{1, 2, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 25, 26, 27, 28, 29, 32, 33, 34, 35, 36}
26


#### Floyd Warshall Algorithm

We will use the Floyd Warshall Algorithm to generate all pairs shortest paths between nodes, this will help us match up odd degree nodes for the CPP problem

more can be found on the Floyd Warshall Algorithm at the following links

[Source 1](https://www.geeksforgeeks.org/floyd-warshall-algorithm-dp-16/)

[Source 2](https://en.wikipedia.org/wiki/Floyd%E2%80%93Warshall_algorithm)

[Source 3](https://www.geeksforgeeks.org/finding-shortest-path-between-any-two-nodes-using-floyd-warshall-algorithm/)

In [1017]:
def fw(graph, costs):
    numNodes = len(nodes)
    
    solutionMatrix = [[0 if i==j else m1.inf for j in range(0,numNodes)] for i in range(0,numNodes)]
    path = [[None for i in range(0, numNodes)] for j in range(0, numNodes)]
    
    for node in graph:
        for adjacentNode in graph[node]:
            solutionMatrix[node][adjacentNode] = costs[(node, adjacentNode)]
            path[node][adjacentNode] = adjacentNode
    
    for k in range(0, numNodes):
        for i in range(0, numNodes):
            for j in range(0, numNodes):
                if solutionMatrix[i][j] > solutionMatrix[i][k] + solutionMatrix[k][j]:
                    solutionMatrix[i][j] = solutionMatrix[i][k] + solutionMatrix[k][j]
                    path[i][j] = path[i][k]
    
    return solutionMatrix, path
                

In [1018]:
shortestPaths, pathTracking = fw(nodes, costs)
oddNodes = genOddNodes(nodes)
# print(shortestPaths)

In [1019]:
def generatePath(shortestPaths, pathTracking, start, end):
    root = start
    end = end
    current = start
    path = [root]
    
    while current != end:
        current = pathTracking[current][end]
        path.append(current)
        
    cost = shortestPaths[root][end]
    return path, cost

path, cost = generatePath(shortestPaths, pathTracking, 0, 30)
print(path, cost)

[0, 37, 36, 35, 34, 5, 6, 7, 22, 30] 69


#### Graph Functions To Support Graph Augmentation and Fleury's Algorithm

In [1020]:
def deleteEdge(start, finish, graph):
    for i in range(0, len(graph[start])):
        if graph[start][i] == finish:
            graph[start].pop(i)
            break

    for i in range(0, len(graph[finish])):
        if graph[finish][i] == start:
            graph[finish].pop(i)
            break
    
    return graph

def addEdge(start, finish, graph):
    if start in graph:
        graph[start].append(finish)
        if finish in graph:
            graph[finish].append(start)
        else:
            graph[finish] = [start]
            
    else:
        graph[start] = [finish]
        if finish in graph:
            graph[finish].append(start)
        else:
            graph[finish] = [start]
        
    return graph

#### Minimum Cost Pairwise Matching

Blossom Algorithm

Use linear programming

Minimize $\sum_{(i,j) \in A}C_{i,j}*X_{i,j}$

$s.t. \sum_{j \in N}X_{i,j} = 1 \forall i \in N$
           
$X_{i,j} \in {0,1} \forall (i,j) \in A$

In [1021]:
def minimizeEdgeSum(shortestPaths, oddNodes):
    oddNodesList = list(oddNodes)

    mdl = Model('Matching')
    mdl.modelSense = GRB.MINIMIZE

    var = range(len(oddNodesList))
    allNodes = [i for i in range(0,len(shortestPaths))]

    x = mdl.addVars(var, var, vtype=GRB.BINARY, name="Pairings")

    mdl.setObjective(quicksum(x[i,j] * shortestPaths[oddNodesList[i]][oddNodesList[j]] for i in var for j in var))

    mdl.addConstrs(quicksum(x[i,j]+x[j,i] for j in var) == 1 for i in var)
    mdl.addConstrs(x[i,j] == 0 for j in var for i in var if j <= i)

    mdl.optimize()
    results = []
    
    for v in mdl.getVars():
        if v.x == 1:
            results.append(v.varName)
    
    newResults = []
    for pair in results:
        temp = pair[9:-1].split(',')
        n1 = oddNodesList[int(temp[0])]
        n2 = oddNodesList[int(temp[1])]
        newResults.append((n1, n2))

    results = newResults
    
    minimumCostPairing = results
    
    return minimumCostPairing 
    
minCost = minimizeEdgeSum(shortestPaths, oddNodes)
print(minCost)

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-8279U CPU @ 2.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 377 rows, 676 columns and 1677 nonzeros
Model fingerprint: 0x51214960
Variable types: 0 continuous, 676 integer (676 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [3e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 570.0000000
Presolve removed 351 rows and 351 columns
Presolve time: 0.00s
Presolved: 26 rows, 325 columns, 650 nonzeros
Variable types: 0 continuous, 325 integer (325 binary)

Root relaxation: objective 1.080000e+02, 23 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  108.00000    0    6  570.00000  108

In [1022]:
def buildMinGraph(nodes, shortestPaths, pairs, costs):
    new = nodes
    for pair in pairs:
        node1, node2 = pair
        new = addEdge(node1, node2, new)
        costs[(node1, node2)] = shortestPaths[node1][node2]
        costs[(node2, node1)] = shortestPaths[node2][node1]

    return new

# res = buildMinGraph(nodes, shortestPaths, minCost, costs)

## Generating Eulerian Path

##### Fleury's Algorithm

Use Fleury's Algorithm to implement Eulerian Cycle Path

[Source 1](https://www.geeksforgeeks.org/fleurys-algorithm-for-printing-eulerian-path/)

In [1023]:
def dfsCount(start, visited, graph):
    visited[start] = True
    count = 1
    
    for adjacent in graph[start]:
        if visited[adjacent] == False:
            count += dfsCount(adjacent, visited, graph)
    return count

def validNextEdge(start, finish, nodes):
    if len(nodes[start]) == 1:
        return True
    else:
        visited = [False for i in range(0, len(nodes))]
        count1 = dfsCount(start, visited, nodes)
        
        nodes = deleteEdge(start, finish, nodes)
        
        visited = [False for i in range(0, len(nodes))]
        count2 = dfsCount(start, visited, nodes)
        
        nodes = addEdge(start, finish, nodes)
        
        if count1 > count2:
            return False
        else:
            return True
        
def fleuryHelper(vertex, nodes, path):
    for adjacentNode in nodes[vertex]:
        if validNextEdge(vertex, adjacentNode, nodes):
            nodes = deleteEdge(vertex, adjacentNode, nodes)
            path.append((vertex, adjacentNode))
            fleuryHelper(adjacentNode, nodes, path)
            
    return path
    
def fleurys(nodes):
    startVertex = 0
    
    for node in nodes:
        if len(nodes[node]) % 2 != 0:
            startVertex = node
            break
        
    return fleuryHelper(startVertex, nodes, [])

In [1024]:
demo1 = {0: [1,2], 1: [0,2], 2: [0,1,3], 3: [2]}
path1 = fleurys(demo1)
print(path)

demo2 = {0:[1,2], 1:[0,2], 2:[1,0]}
path2 = fleurys(demo2)
print(path2)

demo3 = {0:[1,2,3], 1:[0,2,3], 2:[0,1,3,4], 3:[0,4,2,1], 4:[3,2]}
path3 = fleurys(demo3)
print(path3)

[0, 37, 36, 35, 34, 5, 6, 7, 22, 30]
[(0, 1), (1, 2), (2, 0)]
[(0, 1), (1, 2), (2, 0), (0, 3), (3, 4), (4, 2), (2, 3), (3, 1)]


Rural CPP Example

InstaCart type example

#### CPP Algorithm

In [1025]:
def cpp(nodes, costs, shortestPaths):
    oddNodes = genOddNodes(nodes)
    
    if len(oddNodes) == 0:
        return fleurys(nodes)
    
    else:
        pairwiseMatching = minimizeEdgeSum(shortestPaths, oddNodes)
        newGraph = buildMinGraph(nodes, shortestPaths, pairwiseMatching, costs)
        
        result = fleurys(newGraph)
    
        return result, pairwiseMatching

res, match = cpp(nodes, costs, shortestPaths) 

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-8279U CPU @ 2.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 377 rows, 676 columns and 1677 nonzeros
Model fingerprint: 0x51214960
Variable types: 0 continuous, 676 integer (676 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [3e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 570.0000000
Presolve removed 351 rows and 351 columns
Presolve time: 0.00s
Presolved: 26 rows, 325 columns, 650 nonzeros
Variable types: 0 continuous, 325 integer (325 binary)

Root relaxation: objective 1.080000e+02, 23 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  108.00000    0    6  570.00000  108

In [1039]:
def fullPath(res, match, shortestPaths, pathTracking, costs):
    fullPath = []
    duplicateEdges = []
    totalCost = 0
    additionalCost = 0
    checker = set(match)
    for edge in res:
        if edge not in checker:
            fullPath.append(edge)
            totalCost += costs[edge]
        else:
            path, cost = generatePath(shortestPaths, pathTracking, edge[0], edge[1])
            totalCost += cost
            additionalCost += cost
            for i in range(0,len(path)-1):
                fullPath.append((path[i], path[i+1]))
                duplicateEdges.append((path[i], path[i+1]))

    return fullPath, duplicateEdges, totalCost, additionalCost/2

finalPath, duplicateEdges, totalCost, duplicateCost = fullPath(res, match, shortestPaths, pathTracking, costs)

In [1040]:
print(finalPath)
print(len(finalPath))

[(0, 1), (1, 2), (2, 3), (3, 4), (4, 16), (16, 17), (17, 18), (18, 19), (19, 20), (20, 21), (21, 29), (29, 28), (28, 27), (27, 17), (17, 16), (16, 15), (15, 26), (26, 27), (27, 26), (26, 25), (25, 24), (24, 11), (11, 10), (10, 9), (9, 10), (10, 1), (1, 2), (2, 12), (12, 11), (11, 12), (12, 13), (13, 25), (25, 13), (13, 14), (14, 15), (15, 14), (14, 3), (3, 36), (36, 35), (35, 34), (34, 33), (33, 32), (32, 31), (31, 8), (8, 23), (23, 22), (22, 21), (21, 20), (20, 6), (6, 33), (33, 34), (34, 5), (5, 18), (18, 19), (19, 28), (28, 29), (29, 30), (30, 22), (22, 7), (7, 8), (8, 7), (7, 32), (32, 7), (7, 6), (6, 5), (5, 4), (4, 35), (35, 36), (36, 37), (37, 0)]
70


In [1041]:
print(totalCost)

762


In [1042]:
print(duplicateCost)

66.0


In [1043]:
print(duplicateCost/(totalCost-duplicateCost))

0.09482758620689655


$Total Cost: 762$ feet 

$Duplicate Cost: 117$ feet

$Percent Increase: 18.1\%$

There are 57 edges in the graph, 

In [1050]:
accountedFor = set()
doubles = []
for edge in finalPath:
    node1, node2 = edge
    if (node1, node2) in accountedFor or (node2, node1) in accountedFor:
        doubles.append(edge)
    else:
        accountedFor.add(edge)
        
print(len(doubles))

14


In [1051]:
print(doubles)

[(17, 16), (27, 26), (9, 10), (1, 2), (11, 12), (25, 13), (15, 14), (21, 20), (33, 34), (18, 19), (28, 29), (8, 7), (32, 7), (35, 36)]


In [1052]:
addCost = 0
for edge in doubles:
    cost = shortestPaths[edge[0]][edge[1]]
    addCost += cost
print(addCost)

117


In [1053]:
print(addCost/(totalCost-addCost))

0.1813953488372093
